In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import pandas as pd

# download_url = 'https://drive.google.com/uc?id=1tY-4xg6m_dv6IaVPIcd4oC1bK1lW5Tr9&export=download&confirm=t'
# df0 = pd.read_csv(download_url, compression='gzip')
df0 = pd.DataFrame(data=[], columns=['event', 'timestamp', 'user_id'])
df0

,event,timestamp,user_id


In [3]:
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.eventstream.eventstream import Eventstream
from src.graph.p_graph import PGraph, EventsNode
from src.data_processors_lib.rete import CollapseLoops, CollapseLoopsParams
from src.data_processors_lib.rete import DeleteUsersByPathLength, DeleteUsersByPathLengthParams
from src.data_processors_lib.rete import FilterEvents, FilterEventsParams
from src.data_processors_lib.rete import GroupEvents, GroupEventsParams
from src.data_processors_lib.rete import NewUsersEvents, NewUsersParams
from src.data_processors_lib.rete import SplitSessions, SplitSessionsParams
from src.data_processors_lib.rete import StartEndEvents, StartEndEventsParams
from src.data_processors_lib.rete import TruncatePath, TruncatePathParams
from src.data_processors_lib.rete import TruncatedEvents, TruncatedEventsParams
from src.graph.p_graph import PGraph, EventsNode
import inspect


raw_data_schema = RawDataSchema(
    event_name='event', 
    event_timestamp='timestamp', 
    user_id='user_id'
)

stream = Eventstream(
    raw_data=df0,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=stream)

TARGET_EVENT = 'finances/deposit/<payment_name>/success'

def users_with_target_event(df, schema):
    target_users = df[df['event_name'] == TARGET_EVENT]['user_id'].unique()
    return df['user_id'].isin(target_users)

def first_session_filter(df, schema):
    return df['session_id'].str.endswith('_1')

def new_and_not_truncated_users(df, schema):
    truncated_users = df[(df['event_name'] == 'truncated_right')]['user_id'].unique()
    new_users = df[(df['event_name'] == 'new_user')]['user_id'].unique()
    target_users = np.setdiff1d(new_users, truncated_users)
    return df['user_id'].isin(target_users)

def first_session_filter(df, schema):
    return df['session_id'].str.endswith('_1')


node1 = EventsNode(StartEndEvents(params=StartEndEventsParams(**{})))
node2 = EventsNode(NewUsersEvents(params=NewUsersParams(new_users_list="all")))
node3 = EventsNode(TruncatedEvents(params=TruncatedEventsParams(right_truncated_cutoff=(12, 'D'))))
node4 = EventsNode(FilterEvents(params=FilterEventsParams(filter=new_and_not_truncated_users)))
node5 = EventsNode(SplitSessions(params=SplitSessionsParams(
    session_cutoff=(1, 'h'),
    session_col='session_id'
)))
node6 = EventsNode(FilterEvents(params=FilterEventsParams(filter=first_session_filter)))

node7 = EventsNode(FilterEvents(params=FilterEventsParams(filter=users_with_target_event)))
node8 = EventsNode(TruncatePath(params=TruncatePathParams(drop_after=TARGET_EVENT)))
node9 = EventsNode(FilterEvents(params=FilterEventsParams(filter=first_session_filter)))


graph.add_node(node=node1, parents=[graph.root])
graph.add_node(node=node2, parents=[node1])
graph.add_node(node=node3, parents=[node2])
graph.add_node(node=node4, parents=[node3])
graph.add_node(node=node5, parents=[node4])

graph.add_node(node=node6, parents=[node5])
graph.add_node(node=node7, parents=[node6])

graph.add_node(node=node8, parents=[node5])
graph.add_node(node=node9, parents=[node8])



/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable filter was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable negative_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable positive_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()


In [9]:
from src.backend.callback import list_dataprocessor, list_dataprocessor_mock

graph.display()

In [8]:
graph.save_err = None